# LDA - RF Pipeline 

## Import necessary packages

In [1]:
import time
import optuna
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
# from bitermplus import BTM

from dataset.dataset import Dataset
from constants import CLEANED_DATASET_PATH

## Constants

In [2]:
RANDOM_SEED = 0

## Load the Hate Speech Filipino dataset from Hugging Face

In [3]:
dataset = Dataset()
X, Y = dataset.load_from_file(CLEANED_DATASET_PATH)

Data loaded from dataset/cleaned_dataset.pkl


In [4]:
# Re-split the dataset into training, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Vectorize the texts to be able to perform LDA

In [5]:
# Define the CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X_train_vector = vectorizer.fit_transform(X_train)
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

## Perform Latent Dirichlet Allocation on the training set

In [6]:
N_TOPICS = 10
print(f"Performing Latent Dirichlet Allocation for {N_TOPICS} topics")
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = lda.fit_transform(X_train_vector)
X_val_topics = lda.transform(X_val_counts)
X_test_topics = lda.transform(X_test_counts)
print(f"Done performing Latent Dirichlet Allocation for {N_TOPICS} topics")

Performing Latent Dirichlet Allocation for 10 topics
Done performing Latent Dirichlet Allocation for 10 topics


## Search for the best hyperparameters of the Random Forest model

In [7]:
# Define the objective function for Optuna
def create_objective(X_train, Y_train, X_test, Y_test):
    def objective(trial):
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        rf = RandomForestClassifier(n_estimators=n_estimators, 
                                    random_state=RANDOM_SEED)
        rf.fit(X_train, Y_train)
        Y_pred = rf.predict(X_test)
        score = accuracy_score(Y_test, Y_pred)
        return score
    return objective

In [8]:
# Create a study object
study = optuna.create_study(direction="maximize", study_name=f"LDA_RF_Pipeline")

# Create the study objective
objective = create_objective(X_train_topics, Y_train, X_val_topics, Y_val)

# Execute an optimization
study.optimize(objective, n_trials=20, n_jobs=-1)

# Print the best trial results
print(f"Best Accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2024-04-30 21:52:10,086] A new study created in memory with name: LDA_RF_Pipeline
[I 2024-04-30 21:52:17,109] Trial 0 finished with value: 0.6500541711809318 and parameters: {'n_estimators': 116}. Best is trial 0 with value: 0.6500541711809318.
[I 2024-04-30 21:52:18,732] Trial 9 finished with value: 0.6516793066088841 and parameters: {'n_estimators': 138}. Best is trial 9 with value: 0.6516793066088841.
[I 2024-04-30 21:52:19,855] Trial 10 finished with value: 0.652762730227519 and parameters: {'n_estimators': 161}. Best is trial 10 with value: 0.652762730227519.
[I 2024-04-30 21:52:25,193] Trial 1 finished with value: 0.6446370530877573 and parameters: {'n_estimators': 253}. Best is trial 10 with value: 0.652762730227519.
[I 2024-04-30 21:52:26,817] Trial 3 finished with value: 0.6440953412784398 and parameters: {'n_estimators': 276}. Best is trial 10 with value: 0.652762730227519.
[I 2024-04-30 21:52:37,563] Trial 13 finished with value: 0.6462621885157096 and parameters: {'n_est

Best Accuracy: 0.652762730227519
Best hyperparameters: {'n_estimators': 161}


In [9]:
best_n_estimators = study.best_trial.params["n_estimators"]

rf = RandomForestClassifier(n_estimators=best_n_estimators, random_state=RANDOM_SEED)
rf.fit(X_train_topics, Y_train)
Y_pred = rf.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best LDA-RF Model Accuracy: {score}")

Best LDA-RF Model Accuracy: 0.6497022198159177


In [12]:
N_TOPICS = 20
print(f"Performing Non-negative Matrix Factorization for {N_TOPICS} topics")
nmf = NMF(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = nmf.fit_transform(X_train_vector)
X_val_topics = nmf.transform(X_val_counts)
X_test_topics = nmf.transform(X_test_counts)
print(f"Done performing Non-negative Matrix Factorization for {N_TOPICS} topics")

Performing Non-negative Matrix Factorization for 20 topics
Done performing Non-negative Matrix Factorization for 20 topics


In [13]:
rf = RandomForestClassifier(n_estimators=best_n_estimators, 
                            random_state=RANDOM_SEED)
rf.fit(X_train_topics, Y_train)
Y_pred = rf.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best NMF-RF Model Accuracy: {score}")

Best NMF-RF Model Accuracy: 0.6940985381700054
